Framework for making a input catalog...

1) read in a csv file of objects

2) call tess_ephem for each object to see if it has been observed by TESS

3) if it has, summarize the TESS info

4) output two files...  one lists all the bodies searched, the other is a catalog of the objects observed by TESS

This version has an additional loop to generate large catalogs by chunks rather than all at one go.

In [ ]:
from tess_ephem import ephem
import numpy as np
from astropy.time import Time
import pandas as pd
from itertools import repeat
from datetime import datetime
import math
from time import sleep
from tqdm import notebook

Starting with a list made by doing a search in Horizons small body search interface  (https://ssd.jpl.nasa.gov/sbdb_query.cgi). I don't see any documentation for an API, so you have to select the outputs manually.  To start, I've got the following columns:  PDE, full_name, a, e, i om, w, q, ad, per_y, condition_code, H, rot_per, class.  Whether or not these are the right columns is totally tbd!  Note, currently as long as the input list as a column for primary designation (pdes), the code will work.  The output catalog will just copy over any additional input columns into the output catalog

The current catalog returns a line for every unique combination of asteroid and sector.  (In otherwords, if an asteroid is present in multiple sectors, it will have multiple lines.)

Things to check -- do we need to do the same for cameras?  (Right now the code largely assumes that there is only one camera involved, but it does output the number of cameras (and ccds), so we can check this assumption and revisit later on.

In [27]:
# Want to see how long this all takes...
now = datetime.now().time() # time object
print("now =", now)

now = 10:14:45.133796


In [29]:
master_list = pd.read_csv("horizons_2020_12_15.csv") #may have come back to this.  Suspect the pdes col should be string, not integer as we get further in the list
#master_list = pd.read_csv("shortishlist.csv")
#master_list = pd.read_csv("shortlist.csv")
num_asteroids = master_list.shape[0]
num_asteroids

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,10,17,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


1042723

In [19]:
master_list.columns

Index(['pdes', 'full_name', 'a', 'e', 'i', 'om', 'w', 'q', 'ad', 'per_y',
       'condition_code', 'H', 'rot_per', 'class'],
      dtype='object')

In [30]:
# first off, a function that takes in tess_ephem output, asteroid info, and returns a one row data frame with results

def make_catalog_entry(ephem, asteroid):
    #calc values of interest
    num_of_days = ephem.shape[0]
    asteroid["num_of_days"] = num_of_days
    asteroid["sector"] = ephem.sector[0]
    asteroid["num_cameras"] = ephem.camera.nunique()
    asteroid["camera"] = ephem.camera[0]
    asteroid["num_ccd"] = ephem.ccd.nunique()
    asteroid["first_day"] = Time(ephem.index).jd[0] - 2457000  #constant is to conver to "TESS JD"
    asteroid["first_ccd"] = ephem.ccd[0]
    asteroid["first_column"] = ephem.column[0]
    asteroid["first_row"] = ephem.row[0]
    asteroid["last_day"] = Time(ephem.index).jd[-1] - 2457000
    asteroid["last_ccd"] = ephem.ccd[-1]
    asteroid["last_column"]  = ephem.column[(num_of_days-1)]
    asteroid["last_row"]= ephem.row[(num_of_days-1)]
    asteroid["max_pix_per_hour"] = ephem.pixels_per_hour.max()
    asteroid["min_Vmag"] = ephem.vmag.min()
    asteroid["mean_Vmag"] = ephem.vmag.mean()
    asteroid["max_Vmag"] = ephem.vmag.max()
    asteroid["min_phase_angle"] = ephem.phase_angle.min()
    asteroid["mean_phase_angle"] = ephem.phase_angle.mean()
    asteroid["max_phase_angle"] = ephem.phase_angle.max()
    asteroid["min_sun_distance"] = ephem.sun_distance.min()
    asteroid["mean_sun_distance"] = ephem.sun_distance.mean()
    asteroid["max_sun_distance"] = ephem.sun_distance.max()
    asteroid["min_tess_distance"] = ephem.obs_distance.min()
    asteroid["mean_tess_distance"] = ephem.obs_distance.mean()
    asteroid["max_tess_distance"] = ephem.obs_distance.max()
    
    return asteroid

there are two types of dataframes modified / created by this loop...

the original asteroid list will have a column added for whether or not the object is seen by tess.  this data frame includes a row for every asteroid we query about.

the output catalog will have an entry for each unique combination of asteroid, sector for every object seen by tess.  this data frame only has rows for objects seen by tess.

In this implementation, I'm breaking up the input file into chunks because I keep having time outs as the network gets hinky.  This should simplify picking back up and piecing things back together without missing anything...

In [31]:
false_list = []
false_list.extend(repeat(False, num_asteroids))
master_list["in_Tess"] = false_list

In [55]:
#master_list = asteroid_list.copy()

size_of_chunks = 10000
#size_of_chunks = 30 #smaller size for testing
num_of_chunks = math.ceil(num_asteroids / size_of_chunks)
num_of_chunks = num_of_chunks - 73  #this is for a restart
start = 0
start = 730000 # this is for a restart
end = start + size_of_chunks-1

#on December 23, I've got a tess_ephem error that seems to be specific to 2004 QX2, which is idx=577686
# let's try skipping that chunk and move on...
# also errors out on 2011 UT which is idx=724800
# also errorsout on 2019 AE3 which is idx=1004137

for i in notebook.trange(num_of_chunks, desc="chunk loop"):
    asteroid_list = master_list[start:end].copy()
    asteroid_catalog = []
    
    thiscatalog = "catalog_" + str(start) + "_" + str(end) + ".csv"
    thislist = "list" + str(start) + "_" + str(end) + ".csv"

    for idx, ad in asteroid_list.iterrows(): 
        try:
            tt = ephem(str(ad["pdes"]), verbose=True)
        except:  #yes, this is hacky...  but sometimes the network just needs a minute to recover
            sleep(60)
            tt = ephem(str(ad["pdes"]), verbose=True)
        finally:
            #print(asteroid["pdes"], tt.shape[0])
            if tt.shape[0] == 0:
                continue
            #ad["in_Tess"] = True  #dunno why, but this doesn't modify the value in asteroid_list
            asteroid_list.loc[idx,"in_Tess"] = True
            #this_asteroid = ad.copy() #this works, but only if i use the ad["in_Tess"] = True
            this_asteroid = asteroid_list.loc[idx].copy()
            this_asteroid["num_sectors"] = tt.sector.nunique()
            for sector in tt.sector.unique():
                this_sector = tt.loc[tt["sector"] == sector]
                new_entry = make_catalog_entry(this_sector, this_asteroid.copy())
                asteroid_catalog.append(new_entry)


    asteroid_catalog = pd.DataFrame(asteroid_catalog)
    asteroid_catalog.to_csv(thiscatalog, index=False)
    asteroid_list.to_csv(thislist, index=False)
    
    start = end+1
    end = start + size_of_chunks-1
    if end > num_asteroids:
        end = num_asteroids

ValueError: Latitude angle(s) must be within -90 deg <= angle <= 90 deg, got [-2.20057412e+00 -3.04321000e+00 -3.88298958e+00 -4.72025396e+00
 -5.55535057e+00 -6.38863243e+00 -7.22045753e+00 -8.05118810e+00
 -8.88119000e+00 -9.71083206e+00 -1.05404855e+01 -1.13705231e+01
 -1.22013190e+01 -1.30332477e+01 -1.38666835e+01 -1.47020000e+01
 -1.55394243e+01 -1.63786021e+01 -1.72190341e+01 -1.80602228e+01
 -1.89016731e+01 -1.97428941e+01 -2.05834000e+01 -2.14227207e+01
 -2.22604248e+01 -2.30960807e+01 -2.39292471e+01 -2.47594737e+01
 -2.55863028e+01 -2.64092700e+01 -2.72281885e+01 -2.80440043e+01
 -2.88579566e+01 -2.96712946e+01 -3.04852747e+01 -3.13011584e+01
 -3.21202100e+01 -3.29430252e+01 -3.37675115e+01 -3.45908815e+01
 -3.54103255e+01 -3.62230175e+01 -3.70261220e+01 -3.78168000e+01
 -3.85933562e+01 -3.93586746e+01 -4.01168015e+01 -4.08717978e+01
 -4.16277327e+01 -4.23886776e+01 -4.31587000e+01 -4.39402441e+01
 -4.47292789e+01 -4.55201217e+01 -4.63070590e+01 -4.70843588e+01
 -4.78462832e

In [56]:
idx

1004137

In [ ]:
# Want to see how long this all takes...
now = datetime.now().time() # time object
print("now =", now)

In [ ]:
asteroid_catalog.to_csv("TAC_2020_12_15.csv", index=False)

In [ ]:
asteroid_list.to_csv("asteroid_list_2020_12_15.csv",index=False)

In [57]:
master_list.pdes[idx]

'2019 AE3'

In [54]:
ephem("2011 UT", interpolation_step="12H")

TypeError: ephem() got an unexpected keyword argument 'interpolation_step'